## Set Up Network

**This is the code to produce a sample set for and train a neural network.**
All python source code is in the neural.py file

In [ ]:
%pylab inline
from Chempy.parameter import ModelParameters
a = ModelParameters()

In [ ]:
# First create the training dataset
from Chempy.neural import training_data
# %timeit -r 1 -n 1

The above was run on a faster machine, taking 2 hours, 26 minutes and 28 seconds to calculate a training dataset of length 15625. This is stored in the 'Neural/' folder


In [ ]:
# Now create the datasets for model verification (i.e. hyperparameter constraints)
# and for final testing

from Chempy.neural import verification_and_testing
# %timeit -r 1 -n 1

This was again run on a faster PC. The runtime was 19 minutes and 10 seconds, producing the verif_param_grid, verif_abundances, test_param_grid, test_abundances npy data files

## Hyperparameter Optimization

In [ ]:
## The code below will allow optimization of hyperparameter
%pylab inline
from Chempy.parameter import ModelParameters
a = ModelParameters()

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# To plot neural network error against learning rate for optimization

from Chempy.neural import create_network,neural_errors

#learning_rate = [0.001,0.003,0.005,0.007,0.009] #
learning_rate = 10**np.linspace(-3.5,-2,7)
upper = np.zeros((len(learning_rate),1))
lower = np.zeros((len(learning_rate),1))
median = np.zeros((len(learning_rate),1))

for i,rate in enumerate(learning_rate):
    print("Creating network %d of %d" %(i+1,len(learning_rate)))
    create_network(learning_rate=rate,Plot=False)
    param_error = neural_errors('verif')
    median[i] = np.median(param_error)
    upper[i] = np.percentile(param_error,15.865)
    lower[i] = np.percentile(param_error,100-15.865)
    


Creating network 1 of 7
Training epoch 0 of 5000 complete
Training epoch 1000 of 5000 complete


In [ ]:
plt.clf()
plt.errorbar((learning_rate),median,yerr=[median-lower,upper-median])
plt.ylabel('Median L1 error in network across all elements (dex)')
plt.xlabel('Learning Rate')
plt.title('Optimization of Learning Rate')

## Analysis of Optimal Set
*Now using test dataset*

In [ ]:
# Now create and train the neural network for optimal hyperparameters
%pylab inline
from Chempy.neural import create_network
learning_rate = 0.003
epoch, loss = create_network(learning_rate=learning_rate)

In [ ]:
# Create the corner plot
from Chempy.neural import neural_corner_plot, neural_errors
param_error = neural_errors('test')
print('Mean error is %.5f, with standard deviation %.5f' %(np.median(param_error),np.std(param_error)))
neural_corner_plot('test')

In [ ]:
dat=plt.hist(param_error,bins=50)

In [ ]:
from Chempy.neural import neural_errors,neural_output
param_error = neural_errors('test')
param_grid = np.load('Neural/test_param_grid.npy')
chempy_data = np.load('Neural/test_abundances.npy')
neural_data = []
for i in range(len(param_grid)):
    neural_data.append(neural_output(param_grid[i]))

In [ ]:
## ABUNDANCE COMPARISON

# Using first random datapoint in testset

# Prepare plot
fig = plt.figure(figsize=(30.69,8.27), dpi=100)
plt.clf()
text_size = 15
plt.rc('font', family='serif',size = text_size)
plt.rc('xtick', labelsize=text_size)
plt.rc('ytick', labelsize=text_size)
plt.rc('axes', labelsize=text_size, lw=1.)
plt.rc('lines', linewidth = 1.)
plt.rcParams['ytick.major.pad']='8'
plt.rcParams['text.latex.preamble']=[r"\usepackage{libertine}"]
params = {'text.usetex' : True,'font.size' : 16,'font.family' : 'libertine','text.latex.unicode': True}
ax = fig.add_subplot(111)
abundance_names = []
proto_sun = np.load('Chempy/input/stars/Proto-sun.npy')
for item in proto_sun.dtype.names[:-1]:
    if item != 'Fe':
        abundance_names.append('[%s/Fe]' %(item))
    else:
        abundance_names.append('[Fe/H]')
plt.xticks(np.arange(len(a.element_names)),abundance_names)

plt.plot(neural_data[0],'r',label='Neural')
plt.plot(chempy_data[0],'g',label='Chempy')
plt.legend()
plt.ylabel("Abundances relative to solar in dex")
plt.xlabel("Element")
plt.title("Abundance plot")
savefig('Neural/test')

**NOTES / TO DO (see other file also)**
- *Change the diagonal to be correct histogram of data - DONE*
- *Color histogram by mean error in that bar - DONE*
- *Add other params - DONE*
- *Sort axis names - DONE*
- SORT axis sizes (see plot_mcmc.py)
- Vectorize neural output calculation
- Add support for corner plot to take either dataset